In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("data.csv", index_col="RID")

In [14]:
data

,age,income,student,credit rating,buys computer
RID,,,,,
1,30,High,Yes,Fair,Yes
2,25,Low,No,Excellent,No
3,32,Medium,No,Fair,Yes
4,47,High,Yes,Fair,Yes
5,52,Low,Yes,Excellent,No
6,23,Low,No,Excellent,Yes
7,45,Medium,Yes,Fair,Yes
8,51,High,No,Excellent,No
9,20,Medium,Yes,Excellent,Yes


In [15]:
attribute_vals = {att: set() for att in data.columns}
for _, row in data.iterrows():
    for att, att_val in row.items():
        if att_val not in attribute_vals[att]:
            attribute_vals[att].add(att_val)

In [16]:
attribute_vals

{'age': {20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  36,
  37,
  38,
  39,
  40,
  41,
  43,
  44,
  45,
  47,
  48,
  50,
  51,
  52,
  55},
 'income': {'High', 'Low', 'Medium'},
 'student': {'No', 'Yes'},
 'credit rating': {'Excellent', 'Fair'},
 'buys computer': {'No', 'Yes'}}

In [38]:
class Node:
    def __init__(self, parent, criterion, criterion_vals=None, discrete=True, leaf=False):
        # branch constructor
        self.parent = parent
        self.is_leaf = leaf
        if self.is_leaf:
            self.label = criterion
        else:
            self.discrete = discrete
            self.criterion = criterion
            self.criterion_vals = criterion_vals
            self.discrete = discrete
            if discrete:
                self.children = {val: None for val in criterion_vals}
            else:
                print("non discrete created")
                self.children = {True: None, False: None}

    def add_child(self, branch, n):
        self.children[branch] = n

    def apply_criterion(self, row):
        if self.discrete:
            return self.children[row[self.criterion]]
        else:
            print(self.criterion)
            print(row[self.criterion])
            return self.children[row[self.criterion] <= self.criterion_vals]

    def __str__(self):
        if self.is_leaf:
            return  f"parent: {self.parent}\n label: {self.label}"
        else:
            return f"parent: {self.parent}\n criterion: {self.criterion}"


In [39]:
def pure_leaf(D, cls):
    cls_val = D.iloc[0][cls]
    # if all tuples in D has same cls_val then its pure
    for _, row in D.iterrows():
        if row[cls] != cls_val:
            return False
    return True

def majority_voting(D, cls):
    cls_vals = {}
    for _, row in D.iterrows():
        cls_val = row[cls]
        if cls_val not in cls_vals:
            cls_vals[cls_val] = 1
        else:
            cls_vals[cls_val] += 1
    return max(cls_vals, key=cls_vals.get)

def compute_info(D, cls): # Entropy
    n = len(D)
    cls_vals = {}
    for _, row in D.iterrows():
        cls_val = row[cls]
        if cls_val not in cls_vals:
            cls_vals[cls_val] = 1
        else:
            cls_vals[cls_val] += 1

    info = 0
    for _, val in cls_vals.items():
        p = float(val/n)
        info += p*np.log2(p + 1e-6)

    return -info

def make_partitions_discrete(D, attribute, attribute_values):
    # Make partitions of D based on each value of attribute
    partitions = {k: None for k in attribute_values[attribute]}
    # print(partitions)
    for _, row in D.iterrows():
        att_val = row[attribute]
        if partitions[att_val] is None:
            partitions[att_val] = row.copy().to_frame().T
        else:
            partitions[att_val] = pd.concat([partitions[att_val], row.copy().to_frame().T], ignore_index=False)

    return partitions

def make_partitions_continous(D, attribute, split_value):
  partitions = {True: None, False: None}
  for _, row in D.iterrows():
    att_val = row[attribute]
    cond = att_val <= split_value
    if partitions[cond] is None:
      partitions[cond] = row.copy().to_frame().T
    else:
      partitions[cond] = pd.concat([partitions[cond], row.copy().to_frame().T], ignore_index=False)

  return partitions

def get_values(D, attribute):
    vals = set()
    for _, row in D.iterrows():
        att_val = row[attribute]
        if att_val not in vals:
            vals.add(att_val)
    return list(vals)

def attribute_selection_method(D, attribute_list, attribute_values, discrete_atts, cls):
    info = compute_info(D, cls)
    max_info_gain = -1e6
    partitioning_att = attribute_list[0]
    final_partitions = {}
    split_val = None
    for attribute in attribute_list:
        info_att = 0
        if attribute in discrete_atts:
            partitions = make_partitions_discrete(D, attribute, attribute_values)
            for partition in partitions.values():
                if partition is not None:
                    info_part = compute_info(partition, cls)
                    info_att += (len(partition)/len(D))*info_part

            gain_A = info - info_att
            if gain_A > max_info_gain:
                # print("{} selected".format(attribute))
                partitioning_att = attribute
                final_partitions = partitions
                max_info_gain = gain_A
        else:
            # For continous-valued
            values = get_values(D, attribute)
            for i in range(1, len(values)):
              info_att = 0
              split_value = (values[i] + values[i-1])/2.0
              partitions = make_partitions_continous(D, attribute, split_value)
              for partition in partitions.values():
                if partition is not None:
                    info_part = compute_info(partition, cls)
                    info_att += (len(partition)/len(D))*info_part

              gain_A = info - info_att
              if gain_A > max_info_gain:
                # print("{} selected".format(attribute))
                partitioning_att = attribute
                final_partitions = partitions
                split_val = split_value
                max_info_gain = gain_A

    return partitioning_att, final_partitions, split_val



def generate_decision_tree(root,
                           D,
                           attribute_list,
                           attribute_values,
                           discrete_atts,
                           cls,
                           multiway = True):

    # print("cur att_list:", attribute_list)

    if pure_leaf(D, cls):
        label = D.iloc[0][cls]
        print("[] pure leaf with label {} created".format(label))
        N = Node(root, label, leaf=True)
        return N

    if len(attribute_list) == 0:

        label = majority_voting(D, cls)
        N = Node(root, label, leaf=True)
        print("[] leaf with label {} created through majority voting".format(label))
        return N

    attribute, partitions, split_val = attribute_selection_method(D, attribute_list, attribute_values, discrete_atts, cls)
    print("{} selected as criterion".format(attribute))

    if split_val is None: # discrete
      N = Node(root, attribute, partitions.keys())
    else: # continous
      N = Node(root, attribute, split_val, discrete=False)

    if attribute in discrete_atts and multiway:
        attribute_list.remove(attribute)

    for condition, outcome in partitions.items():
        if split_val is None:
          print("{} conditional {} branch".format(attribute, condition))
        else:
          if condition:
            print("{} <= {} branch".format(attribute, split_val))
          else:
            print("{} > {} branch".format(attribute, split_val))
        # print(outcome)
        Dj = outcome
        if Dj is None:
            label = majority_voting(D, cls)
            leaf = Node(N, label, leaf=True)
            print("[] leaf with label {} created after exhaustion".format(label))
            N.add_child(condition, leaf)
        else:
            child = generate_decision_tree(N, Dj, attribute_list, attribute_values, discrete_atts, cls)
            N.add_child(condition, child)

    return N


In [40]:
attribute_list = ["age",	"income",	"student",	"credit rating"]
discrete_atts = ["income",	"student",	"credit rating"]
root = generate_decision_tree(None, data, attribute_list, attribute_vals, discrete_atts, "buys computer")

age selected as criterion
non discrete created
age <= 47.5 branch
income selected as criterion
non discrete created
income <= 39.5 branch
[] pure leaf with label Yes created
income <= 39.5 branch
age selected as criterion
non discrete created
age <= 38.5 branch
[] pure leaf with label Yes created
age > 38.5 branch
age selected as criterion
non discrete created
age <= 40.0 branch
[] pure leaf with label No created
age > 40.0 branch
[] pure leaf with label Yes created
income <= 39.5 branch
student selected as criterion
non discrete created
student <= 21.5 branch
age selected as criterion
non discrete created
age <= 25.5 branch
age selected as criterion
non discrete created
age <= 24.5 branch
age selected as criterion
non discrete created
age <= 22.5 branch
age selected as criterion
non discrete created
age <= 21.5 branch
[] pure leaf with label Yes created
age > 21.5 branch
[] pure leaf with label No created
age > 22.5 branch
[] pure leaf with label Yes created
age > 24.5 branch
[] pure 

In [41]:
root.children

{True: <__main__.Node at 0x7f4d383c8b80>,
 False: <__main__.Node at 0x7f4cfc46cc70>}

In [42]:
num_correct = 0

def classify_tuple(root, row):
    cur_node = root
    while not cur_node.is_leaf:
        cur_node = cur_node.apply_criterion(row)
    return cur_node.label

for _, row in data.iterrows():
    label = classify_tuple(root, row[:-1])
    if label == row[-1]:
        num_correct += 1

age
30
income
High


TypeError: '<=' not supported between instances of 'str' and 'float'

In [ ]:
num_correct / len(data)

0.8333333333333334